# Polygonization

In this workshop we will learn how to convert our topological model of the building into a geometric model.

## 0. Initialization

### 0.0. Importing libraries 

In [121]:

# pip install pyvista==0.28.1

In [122]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import trimesh as tm
import boolean_marching_cubes as bmc

context_path = os.path.relpath('../data/context.obj')
context_mesh = tm.load(context_path)

### 0.1. Generate Symmetry Stencils

In [123]:
# example symmetry strings
##########################
# sym_str = [["OO"], ["XP"], ["XN"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
sym_str = [["OO"], ["XX"], ["YY"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX", "YY", "ZZ"]]
# sym_str = [["OO"]]

stencils = bmc.create_symmetry_stencils(sym_str)



### 0.2. Generate lattices for all possible cubes 

In [124]:
# generate binary representation of all the possible cubes
l_bis = bmc.bi_cube_lattices()


## 1. Profiling

### 1.1. Catalogue the profile of all corners

In [125]:
# find all unique corner arrangements based on stencils
corner_profiles = bmc.extract_corner_profiles(stencils, l_bis)


### 1.2. Find unique corner profiles

In [126]:
# stack corner_profiles vertically
cp_stacked = np.vstack(corner_profiles)

# find the unique arrangements of corners
uniq_corner_arang = np.unique(cp_stacked, axis=0)


### 1.3. Construct unique profile latices

In [127]:
# construct lattices for all unique corner profiles
(corner_loc_lattices, corner_neigh_lattices) = bmc.profiles_to_lattices(uniq_corner_arang, stencils)


### 1.4. Visualize unique profiles

In [151]:
pv.set_plot_theme("document")
p = pv.Plotter(notebook=False)

base_lattice = corner_neigh_lattices[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit * 0.5

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = corner_neigh_lattices[f]
    loc = corner_loc_lattices[f]

    # Add the data values to the cell data
    grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="filled")
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=0.6, show_scalar_bar=False, color ="C86F64")
    
    # Add the data values to the cell data
    grid.cell_arrays["corner"] = loc.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="corner")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.3, show_scalar_bar=False, color="F4EE14")

    return

p.add_slider_widget(create_mesh, [1, len(corner_neigh_lattices)], title='Arrangements', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))

p.show(use_ipyvtk=True)

[(-2.8815166382394035, 1.2484403574017255, 1.1551670702831927),
 (0.0, 0.0, 0.0),
 (0.45527889257996035, 0.2640636967169272, 0.8502890649934031)]

### 1.5. Save unique arrangement profiles 

In [129]:
# save all design templates into lattice CSVs
templates_path = os.path.relpath('../data/bmc/bmc_templates1')
bmc.save_design_templates(corner_loc_lattices, corner_neigh_lattices, templates_path)


## 2. Construct the tile-set

### 2.1. Load sub-tile meshes

In [130]:
# load subtile meshes
Facade_meshes = []
for c in range(len(corner_loc_lattices)):
    corner_mesh_path = os.path.relpath('../data/bmc/BMC_DUMMY_TILESET/t_' + f'{c:02}' + '.obj')
    corner_mesh = tm.load(corner_mesh_path)
    Facade_meshes.append(corner_mesh)



### 2.2. Combine sub-tile meshes to create tile meshes

In [131]:
tiles_meshes = bmc.construct_tile_meshes_old(Facade_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices)
#tiles_meshes = construct_tile_meshes(subtile_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices)


### 2.3. Visualize tile meshes

In [154]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh
pv.set_plot_theme("document")
p = pv.Plotter(notebook=False)

base_lattice = l_bis[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit *0.5

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    mesh = tiles_meshes[i]
    lattice = l_bis[i]

    # Add the data values to the cell data
    grid.cell_arrays["cube"] = lattice.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="cube")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.2, show_scalar_bar=False, color="C86F64")

    # adding the meshes
    p.add_mesh(tri_to_pv(mesh), color='#463E5F', name="sphere")

    return

p.add_slider_widget(create_mesh, [0, len(tiles_meshes)], title='Tiles', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))

p.show(use_ipyvtk=True)

  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_42840/1851846467.py", line 28, in create_mesh
    mesh = tiles_meshes[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_42840/1851846467.py", line 28, in create_mesh
    mesh = tiles_meshes[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_42840/1851846467.py", line 28, in create_mesh
    mesh = tiles_meshes[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_42840/1851846467.py", line 28, in create_mesh
    mesh = tiles_meshes[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_42840/1851846467.py", line 28, in create_mesh
    mesh = tiles_meshes[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_42840/1851846467.py", line 28, in create_mesh
    mesh = tiles_meshes[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Loc

### 2.4. Save the tile-set

In [133]:
tiles_path = os.path.relpath('../data/bmc/BMC_DUMMY_256')
bmc.save_tile_meshes(tiles_meshes, l_bis, tiles_path)


## 3. Boolean Marching Cube

### 3.1. Load envelope lattice

In [134]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/Solar_env_79.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
print(envelope_lattice.shape)
padded_env_arr = np.pad(envelope_lattice, 1, mode="constant", constant_values=False)
padded_minbound = envelope_lattice.minbound - envelope_lattice.unit
envelope_lattice = tg.to_lattice(padded_env_arr, minbound=padded_minbound, unit=envelope_lattice.unit)
print(envelope_lattice.shape)

(24, 14, 7)
(26, 16, 9)


In [135]:
# # create the stencil
# s = tg.create_stencil("von_neumann", 1, 1)
# s.set_index([0,0,0], 0)
# p = pv.Plotter(notebook=True)
# # add the sum function to the stencil
# s.function = tg.sfunc.sum 

# # apply the stencil on the lattice
# neighbor_sum = envelope_lattice.apply_stencil(s)

# # remove if 6
# facade_lattice = envelope_lattice * (neighbor_sum <= 5)
# ground_floor_lattice = facade_lattice * 0
# ground_floor_lattice = facade_lattice[:,:,:1] = 1

# ground_floor_lattice = facade_lattice * 0
# ground_floor_lattice[:,:,:1] = 1
# ground_floor_lattice *= facade_lattice

# p = ground_floor_lattice.fast_vis(p)
# p.show(use_ipyvtk=True)


In [136]:
# # Finding the index of the available voxels in avail_lattice
# avail_index = np.array(np.where(facade_lattice == 1)).T

# for vox_1d_ind in avail_index:
#     vox_3d_ind = np.unravel_index(vox_1d_ind, envelope_lattice.shape)
#     VX, VY, VZ = vox_3d_ind


# # Ground Floor
# ground_floor_lattice = facade_lattice * 0
# ground_floor_lattice[:,:,:1] = 1
# #ground_floor_lattice *= facade_lattice

# p = pv.Plotter(notebook=True)
# p = ground_floor_lattice.fast_vis(p)
# p.show(use_ipyvtk=True)

### 3.2. Extract the cube lattice from the envelope lattice

In [137]:
cube_lattice = envelope_lattice.boolean_marching_cubes()

In [138]:
# cube_lattice[7:9, :10, 0] += 256

In [139]:
# cube_lattice[:12, :12, 0]

### 3.3. tile the cube lattice with a tileset

In [140]:

facade_tiles_path1 = os.path.relpath('../data/bmc/BMC_DUMMY_256')


bmc_facademesh = bmc.marching_cube_mesh(cube_lattice, facade_tiles_path1)


### 3.4. Visualize the final mesh

In [141]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=False)

# # adding the meshes
p.add_mesh(tri_to_pv(bmc_facademesh), color='#638275', name="sphere", opacity=1)
# # fast visualization of the lattice
p = envelope_lattice.fast_vis(p)
# p.add_mesh(tri_to_pv(context_mesh), style='surface')
#p.add_mesh(tri_to_pv(bmc_facademesh), style ='surface',opacity=1)
# plotting
p.show(use_ipyvtk=True)

[(680.7454449076029, 141.10004266238133, 54.96150879092372),
 (646.0, 330.0, 8.0),
 (-0.04666339470490443, 0.2732069464958529, 0.960822820285315)]

### 3.5. Save the final mesh

In [142]:
# final_mesh_path = os.path.relpath('../data/final_mesh.obj')

# with open(final_mesh_path, 'w') as file:
#         file.write(tm.exchange.obj.export_obj(bmc_facademesh))

### Credits

In [143]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Polygonization"